# Feature Selection Using AIC

### Overview

Feature Selection using AIC algorithm

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [ ]:
import os
import urllib.request

data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
data_location = os.path.basename(data_url)

if not os.path.exists (data_location):
    print("Downloading : ", data_url)
    urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)

data_location: house-sales-full.csv


In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

house_prices = pd.read_csv(data_location)
house_prices.sample(10)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
7866,7867,8/17/11,216000,2525400010,Single Family,8/1/11,312700,0.72,"300,618.00",1,...,2.50,3,7,2004,0,0,104000,201000,98038,False
3307,3308,4/26/07,652000,1240700020,Single Family,4/1/07,430900,0.99,"658,506.00",1,...,2.50,4,9,1988,0,0,373000,263000,98074,False
21529,21530,1/9/07,756500,7853240190,Single Family,1/1/07,425600,0.98,"773,564.00",1,...,2.50,4,9,2005,0,0,250000,415000,98065,False
88,89,3/19/08,3650000,46100510,Single Family,3/1/08,415100,0.95,"3,826,741.00",1,...,5.50,5,11,2009,0,0,2412000,2698000,98040,False
1597,1598,2/6/07,620000,626710220,Single Family,2/1/07,427800,0.98,"630,725.00",1,...,2.50,3,8,1982,0,2,294000,234000,98077,False
20789,20790,8/28/07,390000,7625703805,Single Family,8/1/07,434600,1.00,"390,538.00",1,...,1.75,3,6,1943,0,0,228000,264000,98136,False
509,510,2/23/07,561724,207700170,Single Family,2/1/07,427800,0.98,"571,441.00",1,...,2.50,3,8,2007,0,0,159000,363000,-1,False
18624,18625,3/27/06,649000,6821102316,Townhouse,3/1/06,392100,0.90,"720,339.00",1,...,2.50,3,9,2005,0,0,135000,390000,98199,True
9097,9098,7/1/06,764000,2804600030,Single Family,7/1/06,407700,0.94,"815,533.00",1,...,2.25,3,8,1931,0,0,358000,489000,98112,False
26835,26836,4/14/11,400000,9826701175,Single Family,4/1/11,318700,0.73,"546,219.00",1,...,1.75,3,7,1900,0,0,417000,165000,98122,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [ ]:
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [ ]:
house_prices.corr()

<ipython-input-4-61669d780222>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  house_prices.corr()


,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [ ]:
input_columns= ['zhvi_px', 'zhvi_idx', 'NbrLivingUnits', 'SqFtLot', 'SqFtTotLiving', 'SqFtFinBasement', 'Bathrooms', 
                'Bedrooms', 'BldgGrade',   'YrBuilt', 'YrRenovated', 'TrafficNoise', 'LandVal', 'ImpsVal', 'ZipCode', 'NewConstruction']

label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice
0,405100,0.93,2,9373,2400,0,3.00,6,7,1991,0,0,70000,229000,98002,False,280000
1,404400,0.93,1,20156,3764,1452,3.75,4,10,2005,0,0,203000,590000,98166,True,1000000
2,425600,0.98,1,26036,2060,900,1.75,4,8,1947,0,0,183000,275000,98166,False,745000
3,418400,0.96,1,8618,3200,1640,3.75,5,7,1966,0,0,104000,229000,98168,False,425000
4,351600,0.81,1,8620,1720,0,1.75,4,7,1948,0,0,104000,205000,98168,False,240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,332400,0.76,1,1161,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,False,374000
27059,339400,0.78,1,1005,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,True,374000
27060,325300,0.75,1,11170,1070,0,1.00,4,6,1971,0,0,92000,130000,98055,False,165000
27061,400600,0.92,1,6223,1345,0,2.00,3,7,1939,0,0,103000,212000,98166,False,315000


In [ ]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 16)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [ ]:
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import  LassoLarsIC
import numpy as np


# algo + data --> model
# algo =  LinearRegression ()
algo = LassoLarsIC(criterion='aic', max_iter=500)

model = algo.fit(x, np.ravel(y))  # <--- training
model


LassoLarsIC()

## Step 5 : Predict

In [ ]:
## TODO Predict on 'x'
predictions = model.predict(x)
predictions

array([322853.64, 818437.37, 496723.34, ..., 125615.74, 319873.78,
       556915.95])

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a.sample(10)

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice,predicted_price
8919,397900,0.91,1,1504,1460,320,3.25,2,8,2006,0,2,135000,316000,98199,False,415000,"430,196.19"
10651,411700,0.95,1,7548,1200,600,2.25,3,6,1954,0,0,111000,190000,98106,False,315000,"301,104.57"
7099,425600,0.98,1,1030,1500,90,3.25,3,8,2006,0,0,123000,246000,-1,True,347950,"416,057.66"
19898,339900,0.78,1,6960,1450,330,1.50,3,7,1950,0,0,156000,136000,98155,False,309000,"250,443.57"
5120,369800,0.85,1,586,1100,320,2.50,3,7,2008,0,0,85000,178000,98106,True,254000,"268,650.76"
5821,411100,0.94,1,3876,2180,910,2.00,5,7,1924,0,1,280000,307000,98105,False,490000,"565,410.29"
2869,396500,0.91,1,15361,2960,0,3.00,5,10,1927,0,0,1544000,996000,98112,False,2000000,"2,249,681.33"
9950,421200,0.97,1,42852,1090,320,0.75,2,5,1950,0,0,116000,89000,98065,False,339000,"247,103.26"
26858,360700,0.83,1,2400,1680,810,1.75,3,7,1922,0,0,206000,344000,98122,False,489950,"459,021.28"
14308,334700,0.77,1,10400,3080,0,2.50,4,8,2010,0,0,224000,316000,-1,True,443900,"492,643.13"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [ ]:
## coefficients
import numpy as np
np.set_printoptions(precision=2, suppress=True)

print ("coefficients: " , model.coef_)
print ("intercept : ", model.intercept_)

coefficients:  [   1.17    0.      0.      0.34   37.02   -8.19    0.      0.   6465.86
  132.96   -7.63    0.      0.94    0.64    0.14    0.  ]
intercept :  -754775.2275176593


In [ ]:
model.get_params()

{'copy_X': True,
 'criterion': 'aic',
 'eps': 2.220446049250313e-16,
 'fit_intercept': True,
 'max_iter': 500,
 'noise_variance': None,
 'normalize': 'deprecated',
 'positive': False,
 'precompute': 'auto',
 'verbose': False}

In [ ]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "coefficient": model.coef_})
coef

,input_column,coefficient
0,zhvi_px,1.16
1,zhvi_idx,0.00
2,NbrLivingUnits,"9,155.40"
3,SqFtLot,0.37
4,SqFtTotLiving,34.90
5,SqFtFinBasement,-3.99
6,Bathrooms,"6,945.50"
7,Bedrooms,"-8,461.43"
8,BldgGrade,"17,659.24"
9,YrBuilt,-194.66


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

## TODO : find R2 by comparing 'y' and 'predictions'
print ("R2 : " , r2_score(y, predictions))

print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.8424833143425134
MSE :  18511677673.415257
RMSE :  136057.62629641625
